# 15.5 自然语言推断：使用注意力
- **目录**
  - 15.5.1 用于NLI的注意力模型
    - 15.5.1.1 注意力计算（Attending）
    - 15.5.1.2 比较
    - 15.5.1.3 聚合
    - 15.5.1.4 整合代码
  - 15.5.2 NLI注意力模型训练和评估
    - 15.5.2.1 读取数据集
    - 15.5.2.2 创建模型
    - 15.5.2.3 训练和评估模型
    - 15.5.2.4 使用模型

- 在 15.4节中介绍了自然语言推断任务和SNLI数据集。
- 鉴于许多模型都是基于复杂而深度的架构，Parikh等人提出用注意力机制解决自然语言推断问题，并称之为“可分解注意力模型”。
- 这使得模型**没有循环层或卷积层**，在SNLI数据集上以更少的参数实现了当时的最佳结果。
- 本节将描述并实现这种基于注意力的自然语言推断方法（使用MLP），如图15.5.1中所述。

<center><img src='../img/nlp-map-nli-attention.svg'></center>
<center>图15.5.1 将预训练GloVe送入基于<b>注意力</b>和<b>MLP</b>的<b>自然语言推断架构</b></center>

- 本节函数与API列表：
  - mlp：分解注意力计算的多层感知机。
  - Attend：软对齐注意力计算，即计算双向软注意力。
  - Compare：比较假设词元与前提词元以及反过来的比较，生成词元级交互特征。
  - Aggregate：求和两组比较向量，然后通过一个多层感知机进行聚合，并获得分类结果，简而言之就是汇总特征并分类。
  - DecomposableAttention：将注意力计算步骤、比较步骤和聚合步骤组合在一起。
  - predict_snli：预测和输出一对前提和假设之间的逻辑关系。

## 15.5.1 用于NLI的注意力模型

- 与保留前提和假设中词元的顺序相比，我们可以 **将一个文本序列中的词元与另一个文本序列中的每个词元对齐，然后比较和聚合这些信息，以预测前提和假设之间的逻辑关系** 。
- 与机器翻译中源句和目标句之间的词元对齐类似，前提和假设之间的词元对齐可以通过注意力机制灵活地完成。
<center><img src='../img/nli-attention.svg'></center>
<center>图15.5.2 利用注意力机制进行自然语言推断</center><br>
- 图15.5.2描述了使用注意力机制的自然语言推断方法。
- 从高层次上讲，它由三个联合训练的步骤组成：<b>对齐</b>、 <b>比较</b>和<b>汇总</b>。

In [1]:
%matplotlib inline
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

### 15.5.1.1 注意力计算（Attending）

- **第一步是将一个文本序列中的词元与另一个序列中的每个词元对齐**。
  - 比如**前提**是“我确实需要睡眠”，**假设**是“我累了”。
  - 由于语义上的相似性，我们不妨将假设中的“我”与前提中的“我”对齐，将假设中的“累”与前提中的“睡眠”对齐。
  - 同样，我们可能希望将前提中的“我”与假设中的“我”对齐，将前提中的“需要”和“睡眠”与假设中的“累”对齐。
  - 请注意，这种对齐是使用加权平均的“软”对齐，其中理想情况下较大的权重与要对齐的词元相关联。
  - 为了便于演示， 图15.5.2以“硬”对齐的方式显示了这种对齐方式。
- 现在，我们更详细地描述使用注意力机制的软对齐。
  - 用$\mathbf{A} = (\mathbf{a}_1, \ldots, \mathbf{a}_m)$和$\mathbf{B} = (\mathbf{b}_1, \ldots, \mathbf{b}_n)$表示前提和假设，其词元数量分别为$m$和$n$，其中$\mathbf{a}_i, \mathbf{b}_j \in \mathbb{R}^{d}$（$i = 1, \ldots, m, j = 1, \ldots, n$）是$d$维的词向量。
  - 对于软对齐，我们将注意力权重$e_{ij} \in \mathbb{R}$计算为：
  $$e_{ij} = f(\mathbf{a}_i)^\top f(\mathbf{b}_j), \tag{15.5.1}$$
  其中函数$f$是在下面的`mlp`函数中定义的多层感知机。输出维度$f$由`mlp`的`num_hiddens`参数指定。


In [2]:
def mlp(num_inputs, num_hiddens, flatten):
    net = []
    net.append(nn.Dropout(0.2))
    net.append(nn.Linear(num_inputs, num_hiddens))
    net.append(nn.ReLU())
    if flatten:
        ## 从第二维开始展平，比如三维张量
        ## 将其第二维的矩阵展平成一维数组
        net.append(nn.Flatten(start_dim=1))
    net.append(nn.Dropout(0.2))
    #前面一层的输出特征数也是num_hiddens，无论是否展平
    net.append(nn.Linear(num_hiddens, num_hiddens))
    net.append(nn.ReLU())
    if flatten:
        net.append(nn.Flatten(start_dim=1))
    ## Sequential里参数既不是数组也不是list，而是简单列举需要串联的模型对象
    return nn.Sequential(*net)

- 值得注意的是，在公式 15.5.1中，$f$分别输入$\mathbf{a}_i$和$\mathbf{b}_j$，而不是将它们一对放在一起作为输入。
- 这种**分解**技巧导致$f$只有$m + n$个次计算（线性复杂度），而不是$mn$次计算（二次复杂度）。
- 对公式15.5.1中的注意力权重进行规范化，我们计算假设中所有词元向量的加权平均值，以获得假设的表示，该**假设与前提**中索引$i$的词元进行**软对齐**：
$$
\boldsymbol{\beta}_i = \sum_{j=1}^{n}\frac{\exp(e_{ij})}{ \sum_{k=1}^{n} \exp(e_{ik})} \mathbf{b}_j. \tag{15.5.2}
$$
- 同样，我们计算**假设中索引为$j$的每个词元与前提词元**的**软对齐**：
$$
\boldsymbol{\alpha}_j = \sum_{i=1}^{m}\frac{\exp(e_{ij})}{ \sum_{k=1}^{m} \exp(e_{kj})} \mathbf{a}_i. \tag{15.5.3}
$$
- 下面，我们定义`Attend`类来计算假设（`beta`）与输入前提`A`的软对齐以及前提（`alpha`）与输入假设`B`的软对齐。


In [3]:
## 注意力计算，即软对齐计算
class Attend(nn.Module):
    ## 100,200
    def __init__(self, num_inputs, num_hiddens, **kwargs):
        super(Attend, self).__init__(**kwargs)
        self.f = mlp(num_inputs, num_hiddens, flatten=False)

    '''
    A和B的形状都是(256,50,100)，即批量大小、序列长度、词嵌入维度
    '''
    def forward(self, A, B):
        # A/B的形状：（批量大小，序列A/B的词元数，embed_size）
        # f_A/f_B的形状：（批量大小，序列A/B的词元数，num_hiddens）
        ## f_A和f_B的形状(256,50,200)
        f_A = self.f(A)
        f_B = self.f(B)
        
        # e的形状：（批量大小，序列A的词元数，序列B的词元数）
        ## 矩阵的批量乘法(256,50,200)@(256,200,50)=(256,50,50)
        e = torch.bmm(f_A, f_B.permute(0, 2, 1))
        
        # beta的形状：（批量大小，序列A的词元数，embed_size），
        # 意味着序列B被软对齐到序列A的每个词元(beta的第1个维度)
        '''
        beta表示a_j 对 b_k 的注意力权重（序列 B → 序列 A）,即将文本序列B的信息聚合于文本序列A，
        在每个矩阵的行上求softmax,结果是(256,50,50)
        张量的小批量乘积结果：(256,50,50)@(256,50,100)=(256,50,100)
        beta实现公式15.5.2计算过程。
        '''
        beta = torch.bmm(F.softmax(e, dim=-1), B)
        
        # alpha的形状：（批量大小，序列B的词元数，embed_size），
        # 意味着序列A被软对齐到序列B的每个词元(alpha的第1个维度)
        '''
        b_k 对 a_j 的注意力权重（序列 A → 序列 B），即将文本序列A的信息聚合于文本序列B，
        本例中A和B的长度一样，所以即便对e进行转置,alpha的结果仍是(256,50,100)。
        alpha实现15.5.3的计算过程。
        '''
        alpha = torch.bmm(F.softmax(e.permute(0, 2, 1), dim=-1), A)
        ## 二者的返回结果：(256,50,100)
        return beta, alpha

### 15.5.1.2 比较

- 在下一步中，我们将一个序列中的词元与与该词元软对齐的另一个序列进行比较。
  - 请注意，在**软对齐中，一个序列中的所有词元（尽管可能具有不同的注意力权重）将与另一个序列中的词元进行比较**。
- 为便于演示， 图15.5.2对词元以**硬的**方式对齐。
  - 例如，上述的“注意力计算”（attending）步骤确定前提中的“need”和“sleep”都与假设中的“tired”对齐，则将对“tired–need sleep”进行比较。
- 在比较步骤中，我们将来自一个序列的词元的连结（运算符$[\cdot, \cdot]$）和来自另一序列的对齐的词元送入函数$g$（一个多层感知机）：
$$\mathbf{v}_{A,i} = g([\mathbf{a}_i, \boldsymbol{\beta}_i]), i = 1, \ldots, m $$
  $$ \mathbf{v}_{B,j} = g([\mathbf{b}_j, \boldsymbol{\alpha}_j]), j = 1, \ldots, n. \tag{15.5.4}$$
  - 在公式15.5.4中，$\mathbf{v}_{A,i}$是指，所有假设中的词元与前提中词元$i$软对齐，再与词元$i$的比较；
  - 而$\mathbf{v}_{B,j}$是指，所有前提中的词元与假设中词元$i$软对齐，再与词元$i$的比较。

- `Compare`类的功能和目的是实现语义比较，将前提（Premise）和假设（Hypothesis）的词元表示与对方序列的对齐结果进行**深度交互**，生成**更具判别性的比较特征**。
  - 将每个词元的原始表示与其软对齐后的上下文表示结合，通过**非线性**变换生成比较向量。
  - 输入：
    - 原始词向量序列 A（前提）和 B（假设）。
    - 软对齐后的表示 $\beta$（序列 B→A 的对齐结果）和 $\alpha$（序列 A→B 的对齐结果）。
  - 输出：
    - 增强后的**比较向量**V_A和V_B，用于后续的**聚合**和**分类**。
- 上述操作的目的在于：
  - (1) 融合本地与全局信息
    - 原始词向量（$\mathbf{a}_i$ 或 $\mathbf{b}_j$）：编码**词本身的语义**。
    - 对齐向量（$\beta_i$ 或 $\alpha_j$）：编码另一序列中相关词元的上下文信息。
    - 比较向量：将两者结合，同时捕捉词元**自身特征**和跨序列的**关联特征**。
  - (2) 增强判别性，通过非线性变换（MLP），模型能够学习：
    - 词元与其对齐上下文之间的语义兼容性（如“喜欢”和“爱好”是正向关联）。
    - 词元之间的矛盾或中性关系（如“喜欢”和“讨厌”是矛盾关系）。
  - (3) 为聚合步骤提供高阶特征，比较向量 V_A 和 V_B 是后续聚合（求和+分类）的直接输入，其质量直接影响分类性能。

In [4]:
class Compare(nn.Module):
    ## (200,200)
    def __init__(self, num_inputs, num_hiddens, **kwargs):
        super(Compare, self).__init__(**kwargs)
        ## MLP的参数(200,200)
        self.g = mlp(num_inputs, num_hiddens, flatten=False)

    def forward(self, A, B, beta, alpha):
        # cat的结果：(256,50,100),(256,50,100)=(256,50,200)
        # MLP计算结果,即V_A和V_B的形状:(256,50,200)@(200,200)=(256,50,200)
        V_A = self.g(torch.cat([A, beta], dim=2))
        V_B = self.g(torch.cat([B, alpha], dim=2))
        ## 二者的返回结果皆是：(256,50,200)
        return V_A, V_B

### 15.5.1.3 聚合

- 现在我们有有两组比较向量$\mathbf{v}_{A,i}$（$i = 1, \ldots, m$）和$\mathbf{v}_{B,j}$（$j = 1, \ldots, n$）。
- 在最后一步中将聚合这些信息以推断逻辑关系。
- 首先求和这两组比较向量：
$$
\mathbf{v}_A = \sum_{i=1}^{m} \mathbf{v}_{A,i}, \quad \mathbf{v}_B = \sum_{j=1}^{n}\mathbf{v}_{B,j}.\tag{15.5.5}
$$
- 接下来将两个求和结果的连结提供给函数$h$（一个多层感知机），以获得逻辑关系的分类结果：
$$
\hat{\mathbf{y}} = h([\mathbf{v}_A, \mathbf{v}_B]).\tag{15.5.6}
$$
- 聚合步骤在以下`Aggregate`类中定义。


In [5]:
class Aggregate(nn.Module):
    ## 参数400,200,3
    def __init__(self, num_inputs, num_hiddens, num_outputs, **kwargs):
        super(Aggregate, self).__init__(**kwargs)
        
        ''' 
        其实此处flatten=True纯粹是防御作用，保证代码的健壮性和一致性。
        也就是说，即便输入的三维张量，也将该数据通过从第二维进行展平后变成二维张量。
        如果输入的是二维张量，则没任何影响，展平结果仍是二维张量，数据没发生任何变化。

        num_inputs, num_hidden分别为：400,200
        '''
        self.h = mlp(num_inputs, num_hiddens, flatten=True)
        ## (200,3)
        self.linear = nn.Linear(num_hiddens, num_outputs)

    def forward(self, V_A, V_B):
        ''' 
        对两组比较向量分别求和。
        V_A,V_B变成：从(256, 50, 200)变成(256,200)。
        其实就是对矩阵纵向求和，也就是每列求和。
        '''
        V_A = V_A.sum(dim=1)
        V_B = V_B.sum(dim=1)
        # 将两个求和结果的连结送到多层感知机中
        '''
        (1) torch.cat([V_A, V_B], dim=1):(256,200);(256,200)=(256,400)
        (2) self.h: (256,400)@(400,200)=(256,200)
            flatten: 
        (3) self.linear: (256,200)@(200,3)=(256,3)
            3就是分类结果。
        '''
        Y_hat = self.linear(self.h(torch.cat([V_A, V_B], dim=1)))
        return Y_hat

---------------
- **说明：Aggregate类代码分析**


In [6]:
## dim=1时求三维张量的和
v = torch.arange(24).reshape(2,3,4)
v, v.shape,v.sum(dim=1),v.sum(dim=1).shape

(tensor([[[ 0,  1,  2,  3],
          [ 4,  5,  6,  7],
          [ 8,  9, 10, 11]],
 
         [[12, 13, 14, 15],
          [16, 17, 18, 19],
          [20, 21, 22, 23]]]),
 torch.Size([2, 3, 4]),
 tensor([[12, 15, 18, 21],
         [48, 51, 54, 57]]),
 torch.Size([2, 4]))

In [7]:
#v1 = torch.arange(24).reshape(3,8)
flat = nn.Flatten(start_dim=1)
v, flat(v).shape, flat(v)

(tensor([[[ 0,  1,  2,  3],
          [ 4,  5,  6,  7],
          [ 8,  9, 10, 11]],
 
         [[12, 13, 14, 15],
          [16, 17, 18, 19],
          [20, 21, 22, 23]]]),
 torch.Size([2, 12]),
 tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
         [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]]))

---------

### 5.5.1.4 整合代码

- 通过将**注意步骤、比较步骤和聚合步骤**组合在一起，定义了可分解注意力模型来联合训练这三个步骤。


In [8]:
class DecomposableAttention(nn.Module):
    ## 前三个参数：18678，embed_size:100，num_hiddens:200
    def __init__(self, vocab, embed_size, num_hiddens, num_inputs_attend=100,
                 num_inputs_compare=200, num_inputs_agg=400, **kwargs):
        super(DecomposableAttention, self).__init__(**kwargs)
        ## 嵌入层：18678, 100
        self.embedding = nn.Embedding(len(vocab), embed_size)
        ## 注意层：100, 200
        self.attend = Attend(num_inputs_attend, num_hiddens)
        ## 比较层: 200, 200
        self.compare = Compare(num_inputs_compare, num_hiddens)
        ## 有3种可能的输出：蕴涵、矛盾和中性
        ## 聚合层：400,200,3 
        self.aggregate = Aggregate(num_inputs_agg, num_hiddens, num_outputs=3)

    def forward(self, X):
        premises, hypotheses = X
        ## 经过嵌入层的前提A:(256,50,100)
        A = self.embedding(premises)
        ## 经过嵌入层的假设B:(256,50,100)
        B = self.embedding(hypotheses)
        ## (256,50,100)
        beta, alpha = self.attend(A, B)
        ## (256,50,200)
        V_A, V_B = self.compare(A, B, beta, alpha)
        ## (256,3)
        Y_hat = self.aggregate(V_A, V_B)
        return Y_hat

----------
- **说明：DecomposableAttention分解注意力计算**
  - 传统注意力机制（如Transformer中的缩放点积注意力）需要直接计算**所有词对**之间的交互（复杂度 $O(mn)$）。
  - DecomposableAttention直接反映了该模型的核心设计思想，即通过**分解注意力计算**（Decomposing Attention）来降低计算复杂度，同时保持对序列间交互的高效建模。
  - 可分解注意力通过以下分解技巧降低复杂度：
    - （1）**分离计算路径**：
      - 对前提（Premise）和假设（Hypothesis）的词向量**分别**通过MLP（函数 $f$）映射，再计算注意力权重。  
      - 原始计算：$e_{ij} = \text{MLP}([a_i; b_j])$ （需 $mn$ 次MLP计算）。  
      - 分解后计算：$e_{ij} = f(a_i)^\top f(b_j)$ （仅需 $m+n$ 次MLP计算）。  
      - **复杂度优化**： 从二次复杂度（$O(mn)$）降至线性复杂度（$O(m+n)$）。
    - **(2) 任务目标的分解**，模型将自然语言推断任务拆解为三个可独立优化的子步骤：
      - **Attend（对齐）**：计算双向软注意力。
      - **Compare（比较）**：生成词元级交互特征。
      - **Aggregate（聚合）**：汇总特征并分类。  
      - 每个步骤可单独调整或替换，体现模块化设计。
---

## 15.5.2 NLI注意力模型训练和评估
- 在SNLI数据集上对定义好的可分解注意力模型进行训练和评估。
- 从读取数据集开始。

### 15.5.2.1 读取数据集
- 使用15.4节中定义的函数下载并读取SNLI数据集。批量大小和序列长度分别设置为$256$和$50$。

In [9]:
'''
（1）SNLI数据集可以先下载解压，
然后将d2l包的torch.py修改即可正常读取。
将d2l的torch.py文件2636行（具体行可能些微差别）修改成如下：
#data_dir = d2l.download_extract('SNLI')
data_dir = r'../data/snli_1.0'
也就是data_dir改成语料数据所在的目录。可以先下载该文件然后解压即可。

（2）训练和测试数据迭代器的形状为:
[[[前提(256,50)],[假设(256,50)]],[标签(256,)]]
最外层是小批量个数，训练数据有2146个，测试数据有39个小批量。
'''
batch_size, num_steps = 256, 50
train_iter, test_iter, vocab = d2l.load_data_snli(batch_size, num_steps)

read 549367 examples
read 9824 examples


-----

- **说明：调用load_data_snli函数的路径问题**
<img src='../img/15_5_1.png' width=800px />

  - 问题：[Errno 22] Invalid argument: '..\\data\\snli_1.0\\Icon\r'
    - 报错的原因是SNLI数据集的压缩文件"snli_1.0.zip"里面有两个路径为“snli_1.0\Icon\r”和“’__MACOSX/snli_1.0/._Icon\r’”的文件，导致无法解析此路径进而导致整个文件无法解压。
  - 解决方法：
    - 解压数据集"snli_1.0.zip”，找到data文件夹，手动把数据集"snli_1.0.zip"解压到当前文件夹。
    - 然后在d2l包源文件torch.py中定位load_data_snli函数。
    - 将该函数的变量num_workers赋值0。
    - 将变量data_dir赋值为数据集解压后的路径，即把data_dir = d2l.download_extract('SNLI')改为data_dir = r' ../data/snli_1.0'。
<img src='../img/15_5_2.png' width=800px />

-----------

In [10]:
## 训练数据共有2146个小批量
len(iter(train_iter))

2146

In [11]:
## 训练数据和测试数据都是由(256,50)的小批量数据组成。
## 序列长度为50
list(iter(test_iter))[0][0][0].shape,list(iter(test_iter))[0][0][1].shape

(torch.Size([256, 50]), torch.Size([256, 50]))

In [12]:
## 测试数据的小批量个数
len(iter(test_iter))

39

In [20]:
## 标签
list(iter(test_iter))[0][1].shape

torch.Size([256])

In [14]:
## 词表有18678个词元
len(vocab)

18678

In [15]:
'''
训练和测试数据1个小批量的构成，是由二维list构成。
第二维两个元素：一个list和一个tensor。
第一个元素的list由两个tensor构成(前提和假设)。

二维数组：[[tensor(256,50),tensor(256,50)],tensor(256)]。
前提和假设放在list里两个张量。
'''
list(iter(test_iter))[0]

[[tensor([[ 353,  617, 1314,  ...,    1,    1,    1],
          [ 353,  617, 1314,  ...,    1,    1,    1],
          [ 353,  617, 1314,  ...,    1,    1,    1],
          ...,
          [   4,   34,   36,  ...,    1,    1,    1],
          [  59,  147,    8,  ...,    1,    1,    1],
          [  59,  147,    8,  ...,    1,    1,    1]]),
  tensor([[  14,  617,   69,  ...,    1,    1,    1],
          [  14,  617,    6,  ...,    1,    1,    1],
          [   4, 1314,  273,  ...,    1,    1,    1],
          ...,
          [  14,   34,    6,  ...,    1,    1,    1],
          [  59,  147,    8,  ...,    1,    1,    1],
          [   4,   26,    8,  ...,    1,    1,    1]])],
 tensor([2, 0, 1, 2, 0, 1, 0, 2, 1, 2, 0, 1, 0, 0, 1, 1, 0, 2, 0, 1, 2, 1, 0, 2,
         2, 1, 0, 0, 2, 1, 1, 0, 0, 0, 2, 1, 2, 0, 1, 0, 2, 1, 1, 0, 2, 2, 1, 0,
         2, 1, 2, 0, 1, 2, 1, 0, 0, 1, 2, 0, 1, 2, 2, 0, 1, 0, 2, 2, 1, 2, 0, 1,
         0, 2, 1, 0, 2, 0, 0, 1, 1, 1, 0, 0, 1, 2, 1, 2, 0, 2, 1, 0, 2, 0,

### 15.5.2.2 创建模型

- 此处使用预训练好的100维GloVe嵌入来表示输入词元。
- 将向量$\mathbf{a}_i$和$\mathbf{b}_j$在公式15.5.1中的维数预定义为100。 
- 公式15.5.1中的函数$f$和公式15.5.4中的函数$g$的输出维度被设置为200。
- 然后创建一个模型实例，初始化它的参数，并加载GloVe嵌入来初始化输入词元的向量。


In [16]:
embed_size, num_hiddens, devices = 100, 200, d2l.try_all_gpus()
## 参数：18678，100，200
net = DecomposableAttention(vocab, embed_size, num_hiddens)
# 注意glove_embedding预训练词向量的词元个数为40万个，TokenEmbedding加载后会变成40万零1个
glove_embedding = d2l.TokenEmbedding('glove.6b.100d')
embeds = glove_embedding[vocab.idx_to_token]
net.embedding.weight.data.copy_(embeds);

In [17]:
## 词表嵌入：(词表个数，嵌入长度)，嵌入长度取决于glove.6b.100d
embeds.shape

torch.Size([18678, 100])

### 15.5.2.3 训练和评估模型

- 现在可以在SNLI数据集上训练和评估模型。


In [19]:
lr, num_epochs = 0.001, 4
trainer = torch.optim.Adam(net.parameters(), lr=lr)
loss = nn.CrossEntropyLoss(reduction="none")
d2l.train_ch13(net, train_iter, test_iter, loss, trainer, num_epochs, devices)

### 15.5.2.4 使用模型
- 最后，定义预测函数，输出一对前提和假设之间的逻辑关系。


In [27]:
#@save
def predict_snli(net, vocab, premise, hypothesis):
    """预测前提和假设之间的逻辑关系"""
    net.eval()
    premise = torch.tensor(vocab[premise], device=d2l.try_gpu())
    hypothesis = torch.tensor(vocab[hypothesis], device=d2l.try_gpu())
    
    '''
    使用包含两个张量的list输入进行预测，两个张量的第一维相同，第二维可以不同。
    即表示前提假设对的1对1的句子，然后句子的长度可以不同。
    但是在模型中会进行统一处理。
    '''
    label = torch.argmax(net([premise.reshape((1, -1)),
                           hypothesis.reshape((1, -1))]), dim=1)
    return 'entailment' if label == 0 else 'contradiction' if label == 1 \
            else 'neutral'

- 我们可以使用训练好的模型来获得对示例句子的自然语言推断结果。


In [28]:
predict_snli(net, vocab, ['he', 'is', 'good', '.'], ['he', 'is', 'very', 'bad', '.'])

'contradiction'

## 小结

* 可分解注意模型包括三个步骤来预测前提和假设之间的逻辑关系：注意力计算、比较和聚合。
* 通过注意力机制，我们可以将一个文本序列中的词元与另一个文本序列中的每个词元对齐，反之亦然。这种对齐是使用加权平均的软对齐，其中理想情况下较大的权重与要对齐的词元相关联。
* 在计算注意力权重时，分解技巧会带来比二次复杂度更理想的线性复杂度。
* 我们可以使用预训练好的词向量作为下游自然语言处理任务（如自然语言推断）的输入表示。